# NHANES

The National Health and Nutrition Examination Survey (NHANES) is a
cross sectional observational study run every 2-3 years by the
United States Centers for Disease Control (CDC).  It collects
extensive demographic and health-related data on a representative
sample of the US population. We will work with a subset of the possible measures that the NHANES survey collects on basic biometric features.

To get introduced to NHANES, watch this video as a class (it's getting a little old, but still a good picture of the survey):

[NHANES Introduction Video](https://youtu.be/MGuojJXWBzA?si=JUBG2epFvVO35O5w)



First we import the libraries that we will be using.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

Now we load the NHANES data from a file.

In [2]:
df = pd.read_csv("https://github.com/UM-Data-Science-101/lab-07/raw/refs/heads/main/nhanes.csv.gz")


Mounted at /content/gdrive


Many biological processes behave differently during development
compared to adulthood.  For this analysis, we will focus on
people of age 18 or greater (RIDAGEYR contains each subject's age in
years).

In [3]:
df = df.loc[df["RIDAGEYR"] >= 18]

We want the gender variable to be categorical, so we will update that here:

In [4]:
df["RIAGENDR"] = df["RIAGENDR"].replace([1.0, 2.0], ["Male","Female"]).astype("category")

## Getting to know the data

The complete NHANES data set measures hundreds of different health indicators. For this lab, we use a subset related to height, weight, and blood pressure.

Print out the data set size and the names of the columns.

<details>

```
print(df.shape)
df.columns
```
</details>

Here is a brief codebook:

* SEQN: A unique number given to each participant
* RIDAGEYR: The participant's age in years
* RIAGENDR: THe participant's gender
* BMXWT: The participant's weight in kilograms
* BMXHT: The participant's height in centimeters
* BMXBMI: The participant's calculated body-mass-index
* BPXSY[1,2,3]: Three systolic blood pressure measurements for the subject

With the exception of `SEQN` and `RIAGENDR`, the other variables in this data set are quantitative in nature. Use the `.describe()` method for DataFrames to learn about typical values for these measures.

<details>

```
df.describe()
```

</details>

While these numerical summaries can be quite useful for quick snapshots, it is also a good idea to graph the complete distributions. Create histograms for `BMXHT` and `BMXWT` (we will look at some of the other variables later).

<details>

```
sb.histplot(data = df, x = "BMXWT")
sb.histplot(data = df, x = "BMXHT")
```

</details>


Looking at these measures, would you say that either of them exhibit **skew** (unequal amounts of variation above and below the center of the distribution)?

To make this more concrete, let's compute some numerical summaries of skew for both measures.

First, let's see how far balanced quantiles are above and below the median of each distribution using **quantile skewness**:

$$QS = \frac{(Q_b - \tilde X) - (\tilde X - Q_a)}{Q_b - Q_a}$$




with the following components:

* $\tilde X$: the median of the data
* $0 < a < 0.5$ and $b = (1 - a)$ are the values for which we will find the balanced quantiles
* and $Q_b$ and $Q_a$ are the respective quantile values

Write a function that computes quantile skewness for a given series and value of $a$. Use the function to compute quantile skewness for `BMXHT` and `BMXWT` at $a = 0.25$ and $a = 0.1$. What do you notice?




<details>

```
def quantile_skewness(s, a):
  m = s.median()
  qa = s.quantile(a)
  qb = s.quantile(1 - a)
  return ((qb - m) - (m - qa)) / (qb - qa)

(quantile_skewness(df["BMXHT"], 0.25),
 quantile_skewness(df["BMXHT"], 0.1))


(quantile_skewness(df["BMXWT"], 0.25),
 quantile_skewness(df["BMXWT"], 0.1))
```

We observe little skew at the 25th and 75th quantile for each, but some right skew for the weight looking farther out in the 90th and 10th quantiles.

</details>


Another measure of skewness looks at how far observations are from the center of the distribution on average using cubed distance. This measure is the **coefficient of skewness**:

$$C = \frac{1}{S_X^3} \frac{1}{n} \sum_{i=1}^n (X_i - \bar X)^3 $$

Where $\bar X$ is the mean value of the data and $S_X$ is the standard deviation.

Conveniently, the `.skew()` method will calculate this for us. Repeat the skew calculations for height and weight using $C$.



<details>

```
df[["BMXHT", "BMXWT"]].skew()
```

We see that again weight is showing some **right skew**.

</details>

When we encounter skew, it can be advantageous to correct for the skew using a **transformation**. Three that we often use with **positive, right skewed data** are

* reciprocal transfromation: $1/X$
* square root transformation: $\sqrt{X}$
* logarithm transformation: $\log(X)$

Implement these three transformations for `BMXWT`. Create new columns in the DataFrame `df` for each transformation. Recompute either the quantile skewness or coefficient of skewness. Do any of the transformations help reduce skew?

<details>

```
df["BMXWT_recip"] = 1 / df["BMXWT"]
df["BMXWT_sqrt"] = np.sqrt(df["BMXWT"])
df["BMXWT_log"] = np.log(df["BMXWT"])

quantile_skewness(df[["BMXWT", "BMXWT_recip", "BMXWT_sqrt", "BMXWT_log"]], 0.1)

df[["BMXWT", "BMXWT_recip", "BMXWT_sqrt", "BMXWT_log"]].skew()

```

For both techniques, the log transformation works the best.



</details>



# Correlation analyses of body size

The Pearson correlation coefficient describes the association betwen
two quantitative variables.  It mainly captures linear association,
but can detect certain types of non-linear association as well.  The
Pearson correlation coefficient is a numerical statistic that is
closely related to the relationship shown in a scatterplot.

Begin by creating a scatter plot of `BMXHT` (height) and `BMXWT` (weight). Use `alpha = 0.4` to help deal with some of the overplotting.

<details>
    
```
sb.scatterplot(data = df, x = "BMXHT", y = "BMXWT", alpha = 0.4)
```
    
</details>

Think a little about what the correlation for this data set is going to be (positive? negative? zero?). Write yourself a note here.

As we discussed in class, an **increasing relationship** is one in which large values of X are paired with large values of Y and small values of X are paried with small values of Y.

Z-scores are one way to describe how large or small something is:

$$Z_i = \frac{X_i - \bar X}{S_X}$$

where $S_X$ is the standard deviation of the $X$ values.

Write a function that converts a series to its Z-score version (there are a few examples in the notes if you want to see one). Call it `zscores` (we will refer to it later).

<details>

```
def zscores(s):
    return((s - s.mean()) / s.std())
```

</details>

Create new columns in the DataFrame `df` that contain the Z-scores for height and weight.

<details>
    
```
df["BMXHT_Z"] = zscores(df["BMXHT"])
df["BMXWT_Z"] = zscores(df["BMXWT"])
```
</details>

Plot the z-scores against each other and add horizontal and vertical bars using `plt.axvline(0)` and `plt.axhline(0)`.

<details>

```
sb.scatterplot(data = df, x = "BMXHT_Z", y = "BMXWT_Z")
plt.axvline(0, color = "black")
plt.axhline(0, color = "black")
```

</details>

In which quadrants do most of these points fall? Recall that if most points fall in the upper right and lower left quadrants, we have a positive/increasing relationship. If we see most in the upper left and lower right, we have a decreasing relationship. And if all points are equally spread out in all quadrants, we would say that there is no relationship at all.

Recall that the correlation of two variables is defined as the average product of Z-scores for the two measures:

$$r_{xy} = \frac{1}{n} \sum_{i=1}^n \frac{X_i - \bar X}{S_X} \cdot \frac{Y_i - \bar Y}{S_Y}$$
where $S_X$ and $S_Y$ are the standard deviations of the two measurements, respectively.

Using your Z-score columns, calculate the correlation coefficient directly (not using the `.corr()` method).


<details>

```
(df["BMXHT_Z"] * df["BMXWT_Z"]).mean()
```

</details>


Interpret this value. What do the sign and magnitude tell you?



We can also use the `.corr()` method. Apply this to a table with the `BMXHT` and `BMXWT` variables to verify your calculations.

<details>
    
```
df[["BMXHT", "BMXWT"]].corr()
```

Not surprisingly, there is a fairly strong positive association between these variables -- people who are taller tend to be heavier than people who are shorter.

</details>

## Regression Modeling of Height and Weight

Linear regression extends the idea of correlation by providing a specific equation that models the relationship between a predictor $X$ and a response $Y$. This equation is of the form $Y = a + bX$, where $b$ is the **slope** and $a$ is the **intercept**.

The key connection is that the slope $b$ of the regression line is directly related to the correlation coefficient $r$. Specifically, the slope $b$ can be calculated using the formula:

$$b = r_{xy} \cdot \frac{s_{y}}{s_{x}}$$

Where $S_{y}$ and $S_{x}$ are the standard deviations of $Y$ and $X$ respectively. To get the intercept, we want the line to pass through the point $(\bar X, \bar Y)$, so we need to pick the intercept so that $\bar Y = a + b \bar X$. In other words,

$$a = \bar Y - b \bar X$$


Say we wanted to predict `BMXWT` (weight) from `BMXHT` (height). The predictor $X$ would be `BMXHT` and the response $Y$ would be `BMXWT`. Calculate the values of $b$ and $a$.

<details>

```
b = df["BMXHT"].corr(df["BMXWT"]) * df["BMXWT"].std() / df["BMXHT"].std()
a = df["BMXWT"].mean() - b * df["BMXHT"].mean()
(b, a)
```

</details>

Compute the average weight for people who are 120cm tall (according to the model). Compute the average weight for people who are 180cm tall (according to the model).


<details>

```
a + b * 120, a + b * 180
```

</details>

We can also use the coefficient $b$ to understand the difference in conditional means for two groups that **differ by 1 on $X$**.

Verify this by computing the conditional mean of weight for a group defined by having a height of 181 centimeters and compare it to your earlier calculation.

<details>

```
a + b * 181
```

</details>

Interpret the value you got for $b$. If we were to compare two groups of people whose heights differ by one centimeter, how much larger would the conditional mean of weights differ? (At least according to this model).

How much would two groups that had a difference of 50cm of height differ in their conditional mean weight?

## Using statsmodels package

Now that we've calculated the coefficients by hand, we will introduce a built-in method to calculate what we need for us. The following example recreates our previous analysis using the statsmodels package.

In [12]:
import statsmodels.api as sm

## Due to missing values, we need to create a temporary table
temp = df[['BMXWT', "BMXHT"]].dropna()
fitted = sm.OLS(temp["BMXWT"], sm.add_constant(temp["BMXHT"])).fit()
ab = fitted.params
ab

,0
const,-73.474733
BMXHT,0.931844


In [14]:
(ab['const'] + ab['BMXHT'] * 120, ab['const'] + ab['BMXHT'] * 180)


(38.34658757366627, 94.25724781426976)

Perform a linear regression using the statsmodel package to model the conditional mean of `BMXBMI` (body mass index) given `BMXHT`. Interpret the result: For a 1cm change in height, how does the conditional mean of BMI change?

<details>

```
## Due to missing values, we need to create a temporary table
temp = df[['BMXHT', "BMXBMI"]].dropna()
fitted = sm.OLS(temp["BMXBMI"], sm.add_constant(temp["BMXHT"])).fit()
ab = fitted.params
ab
```

</details>

## Conditional Correlation

Throughout this course, we have seen that stratifying a dataset into
more homogeneous subsets can reveal more informative and meaningful
insights about the data.

Create a scatterplot to visualize the
relationship between `BMXHT` and `BMXWT` using the `hue` parameter to separate the male and female data.

<details>

```
sb.scatterplot(x="BMXHT", y="BMXWT", hue="RIAGENDR", alpha=0.4, data=df)
```

</details>

 Stratify (use `groupby` on the recorded gender variable `"RIAGENDR"` -- note the lack of the second "E" in "GENDER") and compute the correlation between height and weight within the groups.

<details>

```
df.groupby("RIAGENDR")[["BMXHT", "BMXWT"]].corr()
```

</details>


For which group is the correlation higher? What does this tell us about the relationship between these two factors for men and women?